In [35]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
YOUTUBE_VIDEO = "https://www.youtube.com/watch?v=cdiD-9MMpb0"
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY")

In [36]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo")
parser = StrOutputParser()

chain = model | parser
chain.invoke("What is the meaning of life?")

'The meaning of life is a complex and subjective question that has been pondered by philosophers, theologians, and individuals throughout history. Different people and cultures have different perspectives on the meaning of life, with some believing it is to seek happiness and fulfillment, others believing it is to serve a higher power or spiritual purpose, and still others believing there is no inherent meaning and it is up to each individual to create their own purpose. Ultimately, the meaning of life is a deeply personal and individual question that each person must grapple with and determine for themselves.'

In [37]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the contect below. If you can't answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""    

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Mary's favorite food is pizza.", question="What is Mary's favorite food?")

'Human: \nAnswer the question based on the contect below. If you can\'t answer the question, reply "I don\'t know".\n\nContext: Mary\'s favorite food is pizza.\n\nQuestion: What is Mary\'s favorite food?\n'

In [38]:
import tempfile
import whisper
from pytube import YouTube  

if not os.path.exists("transcription.txt"):
    youtube = YouTube(YOUTUBE_VIDEO)
    audio = youtube.streams.filter(only_audio=True).first()
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("transcription.txt", "w") as f:
            f.write(transcription)

with open("transcription.txt", "r") as f:
    transcription = f.read()




In [39]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_pinecone import PineconeVectorStore

loader = TextLoader("transcription.txt")
text_documents = loader.load()

chain.invoke("Do aliens exist?")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

embeddings = OpenAIEmbeddings()

index_name = "youtube-idx"

pinecone = PineconeVectorStore.from_documents(
    documents = documents, embedding = embeddings, index_name=index_name
)

In [40]:
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | parser
)

chain.invoke("Do aliens exist?")

'Based on the context provided, the discussion revolves around the possibility of intelligent alien civilizations existing in the galaxy or across galaxies. The speakers express beliefs in the existence of trillions of intelligent alien civilizations slowly traveling through space, potentially meeting each other, collaborating, or even going to war. The speakers also discuss the challenges of detecting alien life due to limitations in current technology, such as the inefficiency of Earth-like radio waves for long-distance communication. While the speakers speculate on the existence of aliens and the difficulties in detecting them, no conclusive evidence is presented in the context. Therefore, the answer to the question "Do aliens exist?" based on this context would be "I don\'t know."'